<a href="https://colab.research.google.com/github/cubricmms/ProjectPneumonia/blob/ssd/hope.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
def check_remaining_time():
  import time, psutil
  Start = time.time()- psutil.boot_time()
  Left= 12*3600 - Start
  print('Time remaining for this session is: ', Left/3600)

In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
# Upload kaggle API key file
# uploaded = files.upload()
!mkdir -p /root/.kaggle
!cp  /content/drive/My\ Drive/ssd/kaggle.json /root/.kaggle
!cp  /content/drive/My\ Drive/ssd/labelmap.pbtxt .
!cp  /content/drive/My\ Drive/ssd/ssdconfig.config .
!cp /content/drive/My\ Drive/ssd/model.ckpt* .

In [4]:
!pip uninstall --quiet folium -y
!pip install --quiet folium==0.2.1
!pip install --quiet pydicom tqdm
!pip install --quiet Pygments==2.5.1
!pip install --quiet tensorflow-object-detection-api

     |████████████████████████████████| 35.3MB 96kB/s 
     |████████████████████████████████| 901kB 23.1MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 71kB 8.0MB/s 
     |████████████████████████████████| 583kB 45.7MB/s 
     |████████████████████████████████| 61kB 11.1MB/s 
     |████████████████████████████████| 2.3MB 61.8MB/s 


In [5]:
# Colab library to upload files to notebook
from google.colab import files

# Install Kaggle library
!pip uninstall --quiet kaggle -y
!pip install --quiet --upgrade pip
!pip install --quiet kaggle==1.5.6

     |████████████████████████████████| 1.4MB 39.2MB/s 
     |████████████████████████████████| 58 kB 7.8 MB/s 


In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [7]:
!kaggle competitions download -c rsna-pneumonia-detection-challenge

100% 3.66G/3.66G [01:08<00:00, 46.4MB/s]
100% 3.66G/3.66G [01:08<00:00, 57.4MB/s]


In [0]:
!unzip -o -q rsna-pneumonia-detection-challenge.zip -d data

In [0]:
!git clone https://github.com/tensorflow/models.git --quiet

In [0]:
!find ./models -name "*.pyc" -exec rm {} \;

In [0]:
!apt-get -qq install protobuf-compiler python-pil python-lxml python-tk

In [0]:
!pip install --quiet Cython

In [0]:
%cd ./models/research/
!protoc object_detection/protos/*.proto --python_out=.

In [0]:
# import os
# os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim'
# !python setup.py -q build
# !python setup.py -q install
# !python object_detection/builders/model_builder_test.py

In [0]:
%cd /content/

In [0]:
import tensorflow as tf
import os
import pandas as pd
import pydicom
from tqdm.notebook import tqdm
from io import BytesIO
import numpy as np
import contextlib2
from object_detection.utils import dataset_util
from object_detection.dataset_tools import tf_record_creation_util

In [0]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [0]:
data_dir = os.path.join(os.getcwd(), 'data')
print('Data directory: ', data_dir)
train_dir = os.path.join(data_dir, "stage_2_train_images")

In [0]:
debug = False
def create_tf_example(patientId, boxes):
    height = 1024 # Image height
    width = 1024 # Image width

    path = os.path.join(train_dir, patientId + ".dcm")

    ds = pydicom.dcmread(path)

    filename = bytes(patientId + '.jpg', 'utf-8') # Filename of the image. Empty if image is not from file
    image_format = b'jpeg' # b'jpeg' or b'png'

    encoded_image_data = ds.PixelData[16:]
    if (debug):
        print(encoded_image_data[:3])

    xmins = [] # List of normalized left x coordinates in bounding box (1 per box)
    xmaxs = [] # List of normalized right x coordinates in bounding box
                # (1 per box)
    ymins = [] # List of normalized top y coordinates in bounding box (1 per box)
    ymaxs = [] # List of normalized bottom y coordinates in bounding box
                # (1 per box)

    classes_text = [] # List of string class name of bounding box (1 per box)
    classes = [] # List of integer class id of bounding box (1 per box)

    for box in boxes:
        if not np.isnan(box[0]):
            if (debug):
                print(box)
            classes_text.append(b'pneumonia')
            classes.append(1)
            
            # x-min y-min width height
            xmins.append(box[0] / width)   # store normalized values for bbox
            xmaxs.append((box[0] + box[2]) / width)
            ymins.append(box[1] / height)
            ymaxs.append((box[1] + box[3]) / height)
        else:
            classes_text.append(b'normal/other')
            classes.append(2)

    if (debug):
        print(xmins)
        print(xmaxs)
        print(ymins)
        print(ymaxs)

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_image_data),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [20]:
num_shards=10
output_filebase='train'

with contextlib2.ExitStack() as tf_record_close_stack:
    output_tfrecords = tf_record_creation_util.open_sharded_output_tfrecords(tf_record_close_stack, output_filebase, num_shards)


    train = pd.read_csv(os.path.join(data_dir,"stage_2_train_labels.csv"))
    groups = train.groupby('patientId')

    count = 0

    for patientId in tqdm(train.drop_duplicates('patientId')['patientId']):
        # print('[{c}]processing patientId={p}'.format(c=count,p=patientId))

        boxes = groups.get_group(patientId).drop(columns=['patientId']).values
        tf_example = create_tf_example(patientId, boxes)

        output_shard_index = count % num_shards
        output_tfrecords[output_shard_index].write(tf_example.SerializeToString())

        count += 1

In [0]:
!mv train-00009-of-00010 test-00009-of-00010
!wget http://download.tensorflow.org/models/object_detection/ssd_inception_v2_coco_2018_01_28.tar.gz
!tar -xvzf  ssd_inception_v2_coco_2018_01_28.tar.gz
!rm ssd_inception_v2_coco_2018_01_28.tar.gz
!ln -s "/content/drive/My Drive" "/content/mydrive"
!cp -r /content/ssd_inception_v2_coco_2018_01_28	/content/mydrive/ssd/
!cp /content/models/research/object_detection/legacy/train.py ./train.py

In [0]:
## Remove corrupted tfrecord! Gonna find out the reason later
import tensorflow as tf
import glob
import os
from google.protobuf.json_format import MessageToJson

files =  glob.glob("/content/train-*") 

filesSize = len(files)
cnt = 0 

for filename in files:
    cnt = cnt + 1
    print('checking %d/%d %s' % (cnt, filesSize, filename))
    try:
        # for example in tf.python_io.tf_record_iterator(filename): 
        for example in tf.data.TFRecordDataset(filename):
            tf_example = tf.train.Example.FromString(example) 
            
            jsonMessage = MessageToJson(tf_example)

    except :
        print("removing %s" % filename)
        os.remove(filename)

In [0]:
! python ./train.py   --train_dir=. --pipeline_config_path=ssdconfig.config --logtostderr &

In [0]:
!cp /content/models/research/object_detection/legacy/eval.py ./eval.py

In [0]:
!python eval.py --logtostderr --pipeline_config_path=ssdconfig.config --checkpoint_dir=. --eval_dir=. &

In [0]:
%load_ext tensorboard
%tensorboard --logdir=.

In [0]:
#!cp model.ckpt* /content/drive/My\ Drive/ssd/